In [3]:
import os
import shutil
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
from pathlib import Path
from IPython.core.getipython import get_ipython
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet50, VGG16, MobileNetV2, Xception, EfficientNetB0, DenseNet121
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnetb0_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet121_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout,Dense, BatchNormalization,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report, f1_score, precision_score, recall_score


In [3]:

def split_dataset(source_dir, output_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Splits each image class in the source_dir into train, validation, and test directories.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    train_dir = os.path.join(output_dir, "train")
    val_dir   = os.path.join(output_dir, "validation")
    test_dir  = os.path.join(output_dir, "test")
    
    for dir_path in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
    
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    for class_name in classes:
        class_src = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(class_src) if os.path.isfile(os.path.join(class_src, f))]
        random.shuffle(images)
        
        total = len(images)
        train_end = int(total * train_ratio)
        val_end   = train_end + int(total * val_ratio)
        
        splits = {
            "train": images[:train_end],
            "validation": images[train_end:val_end],
            "test": images[val_end:]
        }
        
        for split, file_list in splits.items():
            class_dst = os.path.join(output_dir, split, class_name)
            if not os.path.exists(class_dst):
                os.makedirs(class_dst)
            for file_name in file_list:
                src_file = os.path.join(class_src, file_name)
                dst_file = os.path.join(class_dst, file_name)
                shutil.copy2(src_file, dst_file)
    print(f"Dataset split into train, validation, and test in '{output_dir}'.")





In [ ]:

def build_model(Pretrained_model):
    x = Pretrained_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(1, activation='sigmoid', dtype='float32')(x)
    return Model(inputs=Pretrained_model.input, outputs=predictions)

def train_and_evaluate_model(model_name, preprocess_function):
    print(f"\nTraining with {model_name}...\n")
    
    if model_name == 'Xception':
        img_h, img_w = 299, 299
    else:
        img_h, img_w = 224, 224

    # Create ImageDataGenerators for each split
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_function)
    val_datagen   = ImageDataGenerator(preprocessing_function=preprocess_function)
    test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_function)
    
    # Define paths to the split dataset directories
    train_dir = os.path.join(split_dataset_dir, "train")
    val_dir   = os.path.join(split_dataset_dir, "validation")
    test_dir  = os.path.join(split_dataset_dir, "test")
    
    # Data generators for each split
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        seed=42,
    )
    
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False,
        seed=42,
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False  ,
    )
    
    class_indices = test_generator.class_indices    
    
    # Load the chosen pretrained model as the base
    if model_name == 'VGG19':
        Pretrained_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'ResNet50':
        Pretrained_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'VGG16':
        Pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'MobileNetV2':
        Pretrained_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'Xception':
        Pretrained_model = Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'EfficientNetB0':
        Pretrained_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'DenseNet121':
        Pretrained_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    else:
        raise ValueError("Invalid model name")
    
    # Freeze the Pretrained_model layers
    Pretrained_model.trainable = False
    model = build_model(Pretrained_model)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model 
    history = model.fit(
        train_generator,
        steps_per_epoch=math.ceil(train_generator.samples / batch_size),
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=math.ceil(val_generator.samples / batch_size),
        callbacks=[early_stopping],
          )
    
    # evaluate on the test set
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)    
    print(f"\n{model_name} Test Loss: {test_loss:.4f}")
    print(f"{model_name} Test Accuracy: {test_accuracy:.4f}")
    
    # Make predictions on the test set
    predictions = model.predict(test_generator, verbose=1)
    y_pred = (predictions > 0.5).astype(int).flatten()
    y_true = test_generator.classes
    
    # Compute performance metrics
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:\n", cm)
    
    f1 = f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    
    print("\nClassification Report:")
    labels = list(class_indices.keys())
    print(classification_report(y_true, y_pred, target_names=labels))
    
    # Compute ROC Curve and AUC
    fpr, tpr, _ = roc_curve(y_true, predictions.flatten()) 
    roc_auc = auc(fpr, tpr)
    
    return {
        'model': model,
        'history': history.history,
        'confusion_matrix': cm,
        'roc': (fpr, tpr, roc_auc),
        'performance': {'accuracy': test_accuracy, 'f1': f1, 'precision': precision, 'recall': recall},
        'labels': labels
    }



def plot_results(results, run_dir):
    model_names = list(results.keys())
    num_models = len(model_names)
    
    # 1. Plot Confusion Matrices
    fig_cm, axes_cm = plt.subplots(1, num_models, figsize=(5*num_models, 4))
    if num_models == 1:
        axes_cm = [axes_cm]
    for ax, name in zip(axes_cm, model_names):
        cm = results[name]['confusion_matrix']
        im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        ax.set_title(f"{name} Confusion Matrix")
        tick_marks = np.arange(len(results[name]['labels']))
        ax.set_xticks(tick_marks)
        ax.set_xticklabels(results[name]['labels'], rotation=45)
        ax.set_yticks(tick_marks)
        ax.set_yticklabels(results[name]['labels'])
        thresh = cm.max() / 2.0
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
        fig_cm.colorbar(im, ax=ax)
    fig_cm.tight_layout(rect=[0, 0, 1, 0.95])
    fig_cm.suptitle("Confusion Matrices", fontsize=16)
    plt.savefig(os.path.join(run_dir, "confusion_matrices.png"))
    plt.close(fig_cm)
    
    # 2. Plot ROC Curves
    fig_roc, axes_roc = plt.subplots(1, num_models, figsize=(5*num_models, 4))
    if num_models == 1:
        axes_roc = [axes_roc]
    for ax, name in zip(axes_roc, model_names):
        fpr, tpr, roc_auc = results[name]['roc']
        ax.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.2f}")
        ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f"ROC Curve - {name}")
        ax.legend(loc="lower right")
    fig_roc.tight_layout(rect=[0, 0, 1, 0.95])
    fig_roc.suptitle("ROC Curves", fontsize=16)
    plt.savefig(os.path.join(run_dir, "roc_curves.png"))
    plt.close(fig_roc)
    
    # 3. Plot Accuracy and Loss Curves for each model
    for name in model_names:
        history = results[name]['history']
        epochs_range = range(1, len(history['accuracy']) + 1)
        fig_model, (ax_acc, ax_loss) = plt.subplots(1, 2, figsize=(12, 5))
        ax_acc.plot(epochs_range, history['accuracy'], marker='o', label='Train Accuracy')
        ax_acc.plot(epochs_range, history['val_accuracy'], marker='x', linestyle='--', label='Validation Accuracy')
        ax_acc.set_title(f"{name} Accuracy")
        ax_acc.set_xlabel('Epoch')
        ax_acc.set_ylabel('Accuracy')
        ax_acc.legend()
        
        ax_loss.plot(epochs_range, history['loss'], marker='o', label='Train Loss')
        ax_loss.plot(epochs_range, history['val_loss'], marker='x', linestyle='--', label='Validation Loss')
        ax_loss.set_title(f"{name} Loss")
        ax_loss.set_xlabel('Epoch')
        ax_loss.set_ylabel('Loss')
        ax_loss.legend()
        
        fig_model.suptitle(f"Accuracy and Loss Curves - {name}", fontsize=16)
        fig_model.tight_layout(rect=[0, 0, 1, 0.93])
        plt.savefig(os.path.join(run_dir, f"{name}_training_curves.png"))
        plt.close(fig_model)
    
    # 4. Overall Performance Comparison Table
    col_labels = ["Model", "Accuracy", "Precision", "Recall", "F1 Score"]
    cell_text = []
    for name in model_names:
        perf = results[name]['performance']
        row = [name,
               f"{perf['accuracy']:.4f}",
               f"{perf['precision']:.4f}",
               f"{perf['recall']:.4f}",
               f"{perf['f1']:.4f}"]
        cell_text.append(row)
    
    fig_table, ax_table = plt.subplots(figsize=(8, len(model_names)*0.8+1))
    ax_table.axis('tight')
    ax_table.axis('off')
    table = ax_table.table(cellText=cell_text, colLabels=col_labels, loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2)
    fig_table.suptitle("Overall Performance Comparison", fontsize=16)
    plt.savefig(os.path.join(run_dir, "performance_table.png"))
    plt.close(fig_table)
    
    # 5. Save training histories to pickle files
    for name in model_names:
        history = results[name]['history']
        with open(os.path.join(run_dir, f"{name}_history.pkl"), "wb") as f:
            pickle.dump(history, f)
    print("Training histories saved for each model.")



In [ ]:

if __name__ == "__main__":
    
    ##### # Paths to dataset directory
    Base_Folder = 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/' 
    original_dataset_dir = f'{Base_Folder}00- The DataSet/Balanced_DataSet'
    split_dataset_dir = f'{Base_Folder}00- The DataSet/Dataset_split'
    ##### # Split the dataset to train, validation and testing 
    # split_dataset(original_dataset_dir, split_dataset_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1)
    
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_dir = os.path.join(f'{Base_Folder}runs_codes', current_time)
    os.makedirs(run_dir, exist_ok=True)
    ################################ Save the Jupiter Note Book in the runs code 
    # Get the current notebook's path
    def get_notebook_path():
        ipython = get_ipython()
        if ipython is None:
            raise RuntimeError("Not running in Jupyter.")
        # VS Code specific (may not work in all environments)
        if '__vsc_ipynb_file__' in ipython.user_ns:
            return Path(ipython.user_ns['__vsc_ipynb_file__'])
        # Fallback or other environments
        try:
            return Path(ipython.startup_scripts[0])
        except:
            raise RuntimeError("Could not determine notebook path.")


    # Get the current notebook's path and name
    notebook_path = get_notebook_path()
    notebook_name = notebook_path.name
    # Create the destination path
    destination_path = os.path.join(run_dir, notebook_name)

    # Copy the notebook to the specified directory
    shutil.copy(notebook_path, destination_path)
    #########################################################################
    batch_size = 64
    epochs = 50
    
    results = {}
    models = {
        'VGG19': vgg19_preprocess,
        'ResNet50': resnet50_preprocess,
        'VGG16': vgg16_preprocess,
        'MobileNetV2': mobilenetv2_preprocess,
        'Xception': xception_preprocess,
        'EfficientNetB0': efficientnetb0_preprocess,
        'DenseNet121': densenet121_preprocess
    }
    
    for model_name, preprocess in models.items():
        results[model_name] = train_and_evaluate_model(model_name, preprocess)
        
        model_path = os.path.join(run_dir, f"{model_name}_results.pkl")
        results[model_name]['model'].save(f"{run_dir}/{model_name}_model.h5")
        print(f"{model_name} model saved.")
        plot_results(results,run_dir)
    
    print(f"All outputs saved to directory: {run_dir}")



Training with VGG19...

Found 6784 images belonging to 2 classes.
Found 848 images belonging to 2 classes.
Found 848 images belonging to 2 classes.


d:\Learning\University of sadat\Grade 4\Semester 2\06- Graduation Project\Ai_Env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 1113s 10s/step - accuracy: 0.6895 - loss: 0.5695 - val_accuracy: 0.7288 - val_loss: 0.5219
Epoch 2/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 1133s 11s/step - accuracy: 0.8828 - loss: 0.2888 - val_accuracy: 0.9422 - val_loss: 0.1756
Epoch 3/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 943s 9s/step - accuracy: 0.9171 - loss: 0.2053 - val_accuracy: 0.9552 - val_loss: 0.1285
Epoch 4/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 942s 9s/step - accuracy: 0.9439 - loss: 0.1564 - val_accuracy: 0.9623 - val_loss: 0.1051
Epoch 5/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 944s 9s/step - accuracy: 0.9587 - loss: 0.1202 - val_accuracy: 0.9646 - val_loss: 0.0957
Epoch 6/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 942s 9s/step - accuracy: 0.9650 - loss: 0.1043 - val_accuracy: 0.9752 - val_loss: 0.0738
Epoch 7/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 942s 9s/step - accuracy: 0.9716 - loss: 0.0865 - val_accuracy: 0.9705 - val_loss: 0.0723
Epoch 8/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 941s 9s/step - accuracy: 0.9708 - loss: 0.0816 - val_

KeyboardInterrupt: 